In [62]:
%matplotlib notebook
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tif

In [59]:
def read_tif_files():
    '''function to read out tif files in directory'''
    usr_in = input('Please give path to where is your data located at:  ')
    os.chdir(file_dir)
    f_list = list(glob.iglob('**/*.tif', recursive=True))
    dark_list = list(glob.iglob('**/*dark*.tif', recursive=True))
    for el in f_list:
        if el in dark_list:
            f_list.remove(el)
    #out_list = [f for f in f_list if f not in dark_list]
    print('number of total files = {}'.format(len(list(f_list))))
    print('number of dark files = {}'.format(len(list(dark_list))))
    
    return list(f_list)

In [61]:
f_list = read_tif_files()
#np.savetxt('texture_filelist.txt',f_list, fmt='%s')

Please give path to where is your data located at:  ~/Desktop/Globus/multimodal/data3_Textured
number of total files = 172
number of dark files = 20
